<a href="https://colab.research.google.com/github/onucharles/generative-models-cms/blob/master/Assignment3_part3_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import auth

auth.authenticate_user()

!pip install pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
# from google.colab import auth
from oauth2client.client import GoogleCredentials
#@title Functions
def upload_to_drive(filename):
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  to_upload = drive.CreateFile({'title': filename})
  to_upload.SetContentFile(filename)
  to_upload.Upload()
  return to_upload

In [0]:
!git clone --single-branch --branch master https://github.com/onucharles/generative-models-cms.git
!wait $!
!mv -v generative-models-cms/* .
!wait $!
!rm -r generative-models-cms
!wait $!

Cloning into 'generative-models-cms'...
remote: Enumerating objects: 197, done.
remote: Counting objects: 100% (197/197), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 197 (delta 118), reused 150 (delta 76), pack-reused 0
Receiving objects: 100% (197/197), 1.43 MiB | 13.00 MiB/s, done.
Resolving deltas: 100% (118/118), done.
renamed 'generative-models-cms/Assignment3_part3_VAE.ipynb' -> './Assignment3_part3_VAE.ipynb'
renamed 'generative-models-cms/classify_svhn.py' -> './classify_svhn.py'
renamed 'generative-models-cms/density_estimation.py' -> './density_estimation.py'
renamed 'generative-models-cms/estimate_jsd_wd.py' -> './estimate_jsd_wd.py'
mv: cannot move 'generative-models-cms/figures' to './figures': Directory not empty
renamed 'generative-models-cms/models.py' -> './models.py'
renamed 'generative-models-cms/README.md' -> './README.md'
renamed 'generative-models-cms/samplers.py' -> './samplers.py'
renamed 'generative-models-cms/score_fid.py' -> './scor

In [0]:
import os
from torchvision.datasets import utils
import numpy as np
from models import VAE2, Elbo_Normal
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision.utils import save_image
import time
import os
import csv
import torchsummary
import io
from contextlib import redirect_stdout
from math import ceil
from util import load_svhn
import VAE_binary

**Set Hyperparameters**

In [0]:
#Hyperparameters
batch_size = 64
lr = 3e-4
epochs = 21
save_interval = 3 #save model every save_interval epochs (None for not saving)
log_interval = 100 #print results every log_interval batches (in addition to every epoch (None for not printing)
seed = 1111
np.random.seed(seed)
torch.random.manual_seed(seed)
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

**Load Data**

In [0]:
train_loader, val_loader, test_loader = load_svhn(os.curdir, batch_size=batch_size)

Using downloaded and verified file: ./train_32x32.mat
Using downloaded and verified file: ./test_32x32.mat


**Create Save Directory**

In [0]:
save_parent_dir = os.curdir
model_folder = "VAE_SVHN_model_"
previous_folders = [f for f in os.listdir(save_parent_dir) if (model_folder in f and os.path.isdir(f))]
if not previous_folders:
    save_dir = os.path.join(save_parent_dir, model_folder + f'{1:03d}')
else:
    last_model = max(previous_folders)
    i = int(last_model.replace(model_folder,"")) + 1
    save_dir = os.path.join(save_parent_dir, model_folder + f'{i:03d}')
os.mkdir(save_dir)

**Create Model**

In [0]:
model = VAE2()
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = Elbo_Normal()

In [0]:
train_samples, _ = next(iter(train_loader))
val_samples, _ = next(iter(val_loader))
random_z = torch.randn((batch_size, 100))
VAE_binary.generate_samples(model, save_dir=save_dir, epoch=0, train_samples=train_samples, val_samples=val_samples, random_z=random_z, model_outputs_logits=False)
VAE_binary.print_model_summary(model, optimizer, save_dir=save_dir, input_size=(3,32,32))

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2423: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0003
    weight_decay: 0
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 30, 30]             896
               ELU-2           [-1, 32, 30, 30]               0
       BatchNorm2d-3           [-1, 32, 30, 30]              64
            Conv2d-4           [-1, 64, 28, 28]          18,496
               ELU-5           [-1, 64, 28, 28]               0
       BatchNorm2d-6           [-1, 64, 28, 28]             128
         AvgPool2d-7           [-1, 64, 14, 14]               0
            Conv2d-8          [-1, 256, 12, 12]         147,712
               ELU-9          [-1, 256, 12, 12]               0
      BatchNorm2d-10          [-1, 256, 12, 12]             512
        AvgPool2d-11            [-1, 256, 6, 6]               0
           Conv2d-12            [-1, 256, 1, 1]

**Train Model**

In [9]:
train_elbos, val_elbos, epoch_time = VAE_binary.train(model, optimizer, train_loader, val_loader, loss_fn, epochs=epochs,
                                           save_dir=save_dir, save_interval=save_interval, log_interval=log_interval, 
                                           model_outputs_logits=False, train_samples=train_samples, val_samples=val_samples, random_z=random_z)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2423: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


----> Epoch 0,	 Batch 0,	 Train ELBO: -3246.28
----> Epoch 0,	 Batch 100,	 Train ELBO: -2976.27
----> Epoch 0,	 Batch 200,	 Train ELBO: -2960.67
----> Epoch 0,	 Batch 300,	 Train ELBO: -2953.92
----> Epoch 0,	 Batch 400,	 Train ELBO: -2949.86
----> Epoch 0,	 Batch 500,	 Train ELBO: -2946.90
----> Epoch 0,	 Batch 600,	 Train ELBO: -2944.49
----> Epoch 0,	 Batch 700,	 Train ELBO: -2942.12
----> Epoch 0,	 Batch 800,	 Train ELBO: -2939.82
----> Epoch 0,	 Batch 900,	 Train ELBO: -2937.90
----> Epoch 0,	 Batch 1000,	 Train ELBO: -2936.19
-> Epoch 0,	 Train ELBO: -2935.69,	 Validation ELBO: -2915.04,	 Max Validation ELBO: -2915.04,	 Epoch Time: 70.20 seconds
----> Epoch 1,	 Batch 0,	 Train ELBO: -2920.60
----> Epoch 1,	 Batch 100,	 Train ELBO: -2918.78
----> Epoch 1,	 Batch 200,	 Train ELBO: -2918.03
----> Epoch 1,	 Batch 300,	 Train ELBO: -2917.28
----> Epoch 1,	 Batch 400,	 Train ELBO: -2916.73
----> Epoch 1,	 Batch 500,	 Train ELBO: -2916.64
----> Epoch 1,	 Batch 600,	 Train ELBO: -2916.24

**Check Disentangled Representation**

In [10]:
random_z_samples_ = torch.randn((64, 100))
epsilon = 1
save_dir_disentangled = os.path.join(save_dir, "disentangled_representation_samples") 
if not os.path.isdir(save_dir_disentangled):
  os.mkdir(save_dir_disentangled)
VAE_binary.generate_random_samples(model, save_dir_disentangled, random_z_samples=random_z_samples_, epoch=20, num_samples=64, latent_size=100,
                            model_outputs_logits=False, samples_per_image=64, generated_random_file="generated_random_samples")
for i in range(100):
  random_z_samples = random_z_samples_
  random_z_samples[:,i] += epsilon
  VAE_binary.generate_random_samples(model, save_dir_disentangled, random_z_samples=random_z_samples, epoch=20, num_samples=64, latent_size=100,
                            model_outputs_logits=False, samples_per_image=64, generated_random_file=f"generated_random_samples_indexperturbed_{i}")

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2423: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


**Interpolation in the Latent Space**

In [11]:
random_z_0 = torch.randn((64, 100))
random_z_1 = torch.randn((64, 100))
alphas = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
VAE_binary.generate_interpolated_samples(model, save_dir, alphas, interpolate_images=False, random_z_0=random_z_0, random_z_1=random_z_1, 
                              epoch=20, num_samples=64, latent_size=100, model_outputs_logits=False)
VAE_binary.generate_interpolated_samples(model, save_dir, alphas, interpolate_images=True, random_z_0=random_z_0, random_z_1=random_z_1, 
                              epoch=20, num_samples=64, latent_size=100, model_outputs_logits=False)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2423: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


In [12]:
!zip -r VAE_SVHN_model_009.zip VAE_SVHN_model_009
upload_to_drive('VAE_SVHN_model_009.zip')

  adding: VAE_SVHN_model_009/ (stored 0%)
  adding: VAE_SVHN_model_009/generated_random_samples_15.png (deflated 0%)
  adding: VAE_SVHN_model_009/generated_val_samples_best.png (deflated 0%)
  adding: VAE_SVHN_model_009/disentangled_representation_samples/ (stored 0%)
  adding: VAE_SVHN_model_009/disentangled_representation_samples/generated_random_samples_indexperturbed_2_epoch_20_numsamples_64_001.png (deflated 0%)
  adding: VAE_SVHN_model_009/disentangled_representation_samples/generated_random_samples_indexperturbed_49_epoch_20_numsamples_64_001.png (deflated 0%)
  adding: VAE_SVHN_model_009/disentangled_representation_samples/generated_random_samples_indexperturbed_13_epoch_20_numsamples_64_001.png (deflated 0%)
  adding: VAE_SVHN_model_009/disentangled_representation_samples/generated_random_samples_indexperturbed_15_epoch_20_numsamples_64_001.png (deflated 0%)
  adding: VAE_SVHN_model_009/disentangled_representation_samples/generated_random_samples_indexperturbed_21_epoch_20_num

GoogleDriveFile({'title': 'VAE_SVHN_model_009.zip', 'mimeType': 'application/zip', 'kind': 'drive#file', 'id': '1WN_P0jj47BlkJBp3B31IBtnPDoXE0o9a', 'etag': '"O-ZzQh-gY65a42kIgC9WL9_4IwE/MTU1NTYxMzgwNTUwMQ"', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1WN_P0jj47BlkJBp3B31IBtnPDoXE0o9a', 'webContentLink': 'https://drive.google.com/uc?id=1WN_P0jj47BlkJBp3B31IBtnPDoXE0o9a&export=download', 'alternateLink': 'https://drive.google.com/file/d/1WN_P0jj47BlkJBp3B31IBtnPDoXE0o9a/view?usp=drivesdk', 'embedLink': 'https://drive.google.com/file/d/1WN_P0jj47BlkJBp3B31IBtnPDoXE0o9a/preview?usp=drivesdk', 'iconLink': 'https://drive-thirdparty.googleusercontent.com/16/type/application/zip', 'labels': {'starred': False, 'hidden': False, 'trashed': False, 'restricted': False, 'viewed': True}, 'copyRequiresWriterPermission': False, 'createdDate': '2019-04-18T18:56:45.501Z', 'modifiedDate': '2019-04-18T18:56:45.501Z', 'modifiedByMeDate': '2019-04-18T18:56:45.501Z', 'lastViewedByMeDate': '2019-04

In [0]:
#!rm -r VAE_SVHN_model_001